API 文件:<br />
	 https://opendata.cwb.gov.tw/dist/opendata-swagger.html?urls.primaryName=openAPI#/%E9%A0%90%E5%A0%B1/get_v1_rest_datastore_F_C0032_001
     
授權碼: <br />
	CWB-61B968D9-68B1-44CD-87D6-3B39404E4A3F

查詢每小時天氣:<br />
    https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=CWB-61B968D9-68B1-44CD-87D6-3B39404E4A3F

測站名稱: <br />
    南區中心

經緯度: <br />
    "lat": "22.994974",
    "lon": "120.196676"

距離學校: 開車15分鐘的距離

資料參數說明檔: <br />
	https://opendata.cwb.gov.tw/opendatadoc/DIV2/A0003-001.pdf 

# 情境 1 : 透過中央氣象局觀測站 API 抓取天氣資料 

In [32]:
# 透過中央氣象局觀測站 API 抓取天氣資料 [B]
import requests
r = requests.get('https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization=CWB-61B968D9-68B1-44CD-87D6-3B39404E4A3F&locationName=%E5%8D%97%E5%8D%80%E4%B8%AD%E5%BF%83')
# 透過中央氣象局觀測站 API 抓取天氣資料 [E]

In [33]:
weather_data = r.json()

In [34]:
weather_data
# 中央氣象局觀測站 [E]

{'success': 'true',
 'result': {'resource_id': 'O-A0003-001',
  'fields': [{'id': 'lat', 'type': 'Double'},
   {'id': 'lon', 'type': 'Double'},
   {'id': 'locationName', 'type': 'String'},
   {'id': 'stationId', 'type': 'String'},
   {'id': 'obsTime', 'type': 'Timestamp'},
   {'id': 'elementName', 'type': 'String'},
   {'id': 'elementValue', 'type': 'String'},
   {'id': 'parameterName', 'type': 'String'},
   {'id': 'parameterValue', 'type': 'String'}]},
 'records': {'location': [{'lat': '22.994974',
    'lon': '120.196676',
    'locationName': '南區中心',
    'stationId': '467410',
    'time': {'obsTime': '2022-01-09 12:20:00'},
    'weatherElement': [{'elementName': 'ELEV', 'elementValue': '13.80'},
     {'elementName': 'WDIR', 'elementValue': '330'},
     {'elementName': 'WDSD', 'elementValue': '4.20'},
     {'elementName': 'TEMP', 'elementValue': '22.70'},
     {'elementName': 'HUMD', 'elementValue': '0.61'},
     {'elementName': 'PRES', 'elementValue': '1012.70'},
     {'elementName': 

# 情境 2 : 利用爬蟲從 codis 網站抓天氣資料

In [6]:
import re #regular expression正則表示法
import pandas as pd #將資料處理成csv
import requests #取得網頁內容
from bs4 import BeautifulSoup
import csv
info = [
    ["臺南","台南市","中西區","8.csv", "https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker="]
]

datelist = pd.date_range(start='2017-01-01', end='2022-05-14')
date = []
for day in datelist:
    date.append(str(day)[:10])
# print(date)

monthData = []
colName = ["測站", "縣市別", "區域", "監測日期", "監測時間(含日期)", "監測時間", \
		   "監測時間（小時）", "測站氣壓", "溫度(℃)", "相對溼度(%)", "風向", "風速(m/s)", "最大陣風(m/s)", \
		   "降水量(mm)", "日照時數(小時)", "紫外線指數"]
url = ""

try:
    for i in range(len(info)):
        for j in range(len(date)):
            #print(i, j)
            #print(info[i][0], info[i][1], info[i][2], info[i][3], info[i][4])
            url = info[i][4] + date[j]
            #print(url)
            request = requests.get(url)
            Webdata = BeautifulSoup(request.text, "html.parser")

            station = info[i][0]
            city = info[i][1]
            region = info[i][2]
            monitorDate = date[j].replace('-', '/')

            dailyData = []
            trCount = 0
            hrIndex = 0
            tdIndex = 0
            rowData = Webdata.find("tbody")  # row

            for z in range(24):
                dailyData.append([])
                dailyData[z].extend([station, city, region, monitorDate])
            for tRow in rowData.select("tr"):
                if trCount >= 3:
                    for tData in tRow.find_all("td"):
                        if tdIndex == 0:
                            if(hrIndex < 10):
                                monitorTimeDate = monitorDate + " " + "0" + str(hrIndex) + ":00:00"
                                monitorTime = "0" + str(hrIndex) + ":00:00"
                                hour = "0" + str(hrIndex)
                            else:
                                monitorTimeDate = monitorDate + " " + str(hrIndex) + ":00:00"
                                monitorTime = str(hrIndex) + ":00:00"
                                hour = str(hrIndex)
                            dailyData[hrIndex].extend([monitorTimeDate, monitorTime, hour])
                        elif tdIndex == 1 or tdIndex == 3 or tdIndex == 5 or tdIndex == 10:
                            dailyData[hrIndex].append(tData.text[:-1])
                        elif tdIndex == 6:
                            tmp = tData.text
                        elif tdIndex == 7:
                            dailyData[hrIndex].append(tData.text[:-1])
                            dailyData[hrIndex].append(tmp[:-1])
                        elif tdIndex == 12 or tdIndex == 8 or tdIndex == 15:
                            if (tData.text[:-1] == '...'):
                                dailyData[hrIndex].append('-')
                            else:
                                dailyData[hrIndex].append(tData.text[:-1])
                        tdIndex += 1
                    hrIndex += 1
                tdIndex = 0
                trCount += 1
            monthData.extend(dailyData)

        print('============ ' + info[i][3] + ' Done ===========')
    print(monthData)
finally:
    print('======= start writing =======')
    with open("天氣觀測資料20220515.csv", 'w', encoding='ANSI', newline='') as donefile:
        writer = csv.writer(donefile)
        writer.writerow(colName)
        for row in monthData:
            writer.writerow(row)
    print('============ Done ===========')

# Codis [E]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# Codis 月份 [B]
import re #regular expression正則表示法
import pandas as pd #將資料處理成csv
import requests #取得網頁內容
from bs4 import BeautifulSoup
import csv

info = [
    ["臺南","台南市","中西區","8.csv", "https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker="]
]

month_list = []
for y in range(2017, 2018):
    for m in range(1, 13):
        m = ("00" + str(m))[-2:]
        month_list.append('{}-{}'.format(y, m))

monthData = []

colName = ["測站", "縣市別", "區域", "監測年月份", "監測時間（天）", 
		   "測站氣壓", "溫度(℃)", "最高溫度(℃)", "最低溫度(℃)", "相對溼度(%)", "風向", "風速(m/s)", \
		   "降水量(mm)", "日照時數(小時)"]
url = ""

try:
    for i in range(len(info)):
        for j in range(len(month_list)):
            #print(i, j)
            #print(info[i][0], info[i][1], info[i][2], info[i][3], info[i][4])
            url = info[i][4] + month_list[j]
            print(url)
            request = requests.get(url)
            Webdata = BeautifulSoup(request.text, "html.parser")

            station = info[i][0]
            city = info[i][1]
            region = info[i][2]
            monitorYearMonth = month_list[j]
            dailyData = []
            trCount = 0
            dayIndex = 0
            tdIndex = 0
            rowData = Webdata.find("tbody")  # row

            for z in range(len(rowData.select("tr"))-3):
                dailyData.append([])
                dailyData[z].extend([station, city, region, monitorYearMonth])

            for tRow in rowData.select("tr"):
                if trCount >= 3:
                    for tData in tRow.find_all("td"):
                        if tdIndex == 0: # 觀測時間 (day)
                            day_str = ("00" + str(dayIndex+1))[-2:]
                            dailyData[dayIndex].extend([str(day_str)])
                        elif tdIndex in [1, 7, 8, 10, 13, 21]: # 測站氣壓, 氣溫, 最高溫, 最低溫, 相對溼度, 降水量
                            dailyData[dayIndex].append(tData.text[:-1])
                        elif tdIndex == 16: # 風速
                            tmp = tData.text
                        elif tdIndex == 17: # 風向
                            dailyData[dayIndex].append(tData.text[:-1])
                            dailyData[dayIndex].append(tmp[:-1])
                        elif tdIndex == 27: # 日照時數
                            if (tData.text[:-1] == '...'):
                                dailyData[dayIndex].append('-')
                            else:
                                dailyData[dayIndex].append(tData.text[:-1])
                        tdIndex += 1
                    dayIndex += 1
                tdIndex = 0
                trCount += 1
            monthData.extend(dailyData)

        print('============ ' + info[i][3] + ' Done ===========')
    print(monthData)
finally:
    print('======= start writing =======')
    with open("月天氣觀測資料20220329.csv", 'w', encoding='ANSI', newline='') as donefile:
        writer = csv.writer(donefile)
        writer.writerow(colName)
        for row in monthData:
            writer.writerow(row)
    print('============ Done ===========')

# Codis [E]


https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker=2017-01
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker=2017-02
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker=2017-03
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker=2017-04
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2597&altitude=40.8m&datepicker=2017-05
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467410&stname=%25E8%2587%25BA%25E5%258D%2